## Unit test for ``import_data``

In [2]:
import hdf5storage
import numpy as np
import pandas as pd


def import_matlab_data(filename):
    """Taking a Matlab file of Arbin data from the CALCE database and imports it into a host of different NumPy arrays
        Filename: Matlab file 
    """
    
    #Importing the file as a HDF5 dataframe, stored in the form of a dictionary.  
    df = hdf5storage.loadmat(filename)
    return df


def initialize_datasets(filename):
    """This function initializes the columns of the dataframe. 
        Filename = matlab file
    """
    
    #Generate the dataframe
    df = import_matlab_data(filename) 
    #Define a variable that contains the names of the dictionary's keys
    keys = df.keys()

    #Store the filename (second key in the dictionary) under the variable call 'data' 
    for i in keys:
        if i == '#subsystem#':
            pass
        else:
            data = i
    #To get the data out of the subsystem, start the count at 2
    count = 2

    for idx in range(1, len(df[data])):
        #Get the time column from the dataset (this gives the length of the experiment)
        time_sum = df['#subsystem#'][0][0][0][count][0][0].flatten()

        #Evaluate the length of the experiment. 
        #If it is a single cycle (less than 3500 data points), ignore it and go to the next experiment  
        if len(time_sum) < 3500:
            count = count + 7
        #If there are more cycles, initialize the names of the columns.
        else:
            datetime_sum = df['#subsystem#'][0][0][0][count][0][1].flatten()
            step_sum = df['#subsystem#'][0][0][0][count][0][2].flatten()
            cycle_sum = df['#subsystem#'][0][0][0][count][0][3].flatten()
            current_sum = df['#subsystem#'][0][0][0][count][0][4].flatten()
            voltage_sum = df['#subsystem#'][0][0][0][count][0][5].flatten()
            charge_ah_sum = df['#subsystem#'][0][0][0][count][0][6].flatten()
            discharge_ah_sum = df['#subsystem#'][0][0][0][count][0][7].flatten()
            count = count + 7
            break
    
    return count, idx, time_sum, datetime_sum, step_sum, cycle_sum, current_sum, voltage_sum, charge_ah_sum, discharge_ah_sum    

def single_pd_matlab_data(filename):
    """This function generates a dataframe with all the cycles included in the matlab file.
        Filename = matlab file
    """

    #Generate the dataframe
    df = import_matlab_data(filename) 
    #Define a variable that contains the names of the dictionary's keys
    keys = df.keys() 

    #Store the filename (second key in the dictionary) under the variable call 'data'
    for i in keys:
        if i == '#subsystem#':
            pass
        else:
            data = i

    #Initialize the name of the columns             
    count, idx, time_sum, datetime_sum, step_sum, cycle_sum, current_sum, voltage_sum, charge_ah_sum, discharge_ah_sum = initialize_datasets(filename)
    
    #Start the count from the 3rd row of the dataset
    start_ind = idx + 1
  
    for idx in range(int(start_ind), len(df[data])):
        #To access the number of cycles in each dataset.
        length_test = (df['#subsystem#'][0][0][0][count][0][0].flatten())
         
        #For missing data: Store the number of cycles with missing data and add them up into the cycles count
        if df[data][idx][2].shape == (0,0):
            name = df[data][idx][0][0][0]
            name = name.split(' ')
            number = float(name[0])
            cycle_sum = np.append(cycle_sum, (cycle_sum[-1] + number))
        #For single cycles: Ignore cycles with less than 3500 values.
        elif len(length_test) < 3500:
            count = count + 7
        #Extract data from each column
        else:
            time_idx = (df['#subsystem#'][0][0][0][count][0][0].flatten()) + time_sum[-1]
            time_sum = np.concatenate((time_sum, time_idx), axis = 0)

            datetime_idx = df['#subsystem#'][0][0][0][count][0][1].flatten()
            datetime_sum = np.concatenate((datetime_sum, datetime_idx), axis = 0)

            step_idx = df['#subsystem#'][0][0][0][count][0][2].flatten()
            step_sum = np.concatenate((step_sum, step_idx), axis = 0)

            #For the cycle number, evaluate if there were missing data.
            if len(cycle_sum) > len(current_sum):
                diff = len(cycle_sum) - len(current_sum)
                cycle_prior = cycle_sum[-1]
                for i in range(0,diff):
                    cycle_sum = np.delete(cycle_sum, -1)
                cycle_idx = (df['#subsystem#'][0][0][0][count][0][3].flatten()) + cycle_prior
                cycle_test = (df['#subsystem#'][0][0][0][count][0][3].flatten())
                cycle_sum = np.concatenate((cycle_sum, cycle_idx), axis = 0)
            else:
                cycle_prior = cycle_sum[-1]
                cycle_idx = (df['#subsystem#'][0][0][0][count][0][3].flatten()) + cycle_prior
                cycle_test = (df['#subsystem#'][0][0][0][count][0][3].flatten())
                cycle_sum = np.concatenate((cycle_sum, cycle_idx), axis = 0)

            current_idx = df['#subsystem#'][0][0][0][count][0][4].flatten()
            current_sum = np.concatenate((current_sum, current_idx), axis = 0)

            voltage_idx = df['#subsystem#'][0][0][0][count][0][5].flatten()
            voltage_sum = np.concatenate((voltage_sum, voltage_idx), axis = 0)

            charge_ah_idx = df['#subsystem#'][0][0][0][count][0][6].flatten()
            charge_ah_sum = np.concatenate((charge_ah_sum, charge_ah_idx), axis = 0)

            discharge_ah_idx = df['#subsystem#'][0][0][0][count][0][7].flatten()
            discharge_ah_sum = np.concatenate((discharge_ah_sum, discharge_ah_idx), axis = 0)

            count = count + 7

            if count > (len(df['#subsystem#'][0][0][0])-7):
                break

    #Creat a DataFrame from the arrays for each column             
    host_df = pd.DataFrame()

    host_df['time'] = time_sum
    host_df['datetime'] = datetime_sum
    host_df['step'] = step_sum
    host_df['cycle'] = cycle_sum
    host_df['current_amp'] = current_sum
    host_df['voltage'] = voltage_sum
    host_df['charge_ah'] = charge_ah_sum
    host_df['discharge_ah'] = discharge_ah_sum
    
    return host_df

/Users/rossana13/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
df2 = hdf5storage.loadmat('converted_PL03.mat')

In [6]:
len(df2.keys())

2

In [15]:
type(df2['#subsystem#'][0][0][0][2][0][0].flatten())


numpy.ndarray

In [8]:
import import_data
import pandas as pd
import numpy as np

def test_import_matlab_data():
    
    df = import_matlab_data('converted_PL03.mat')
    
    assert len(df2.keys()) == 2
    


In [9]:
test_import_matlab_data()

In [11]:
a = np.array([1,1,1])
type(a)

numpy.ndarray

In [29]:
count, idx, time_sum, datetime_sum, step_sum, cycle_sum, current_sum, voltage_sum, charge_ah_sum, discharge_ah_sum = initialize_datasets('converted_PL03.mat')

In [31]:
type(datetime_sum)

numpy.ndarray

In [41]:
import import_data
import pandas as pd
import numpy as np

def test_initialize_datasets():
    
    count, idx, time_sum, datetime_sum, step_sum, cycle_sum, current_sum, voltage_sum, charge_ah_sum, discharge_ah_sum = initialize_datasets('converted_PL03.mat')
    
    assert count == 16
    assert idx == 2
    #The other outputs should be NumPy arrays. Test that with an auxiliar array
    a = np.array([1,1,1]) 
    
    assert type(a) == type(time_sum)
    assert type(time_sum) == type(datetime_sum)
    assert type(time_sum) == type(step_sum)
    assert type(time_sum) == type(cycle_sum)
    assert type(time_sum) == type(current_sum)
    assert type(time_sum) == type(voltage_sum)
    assert type(time_sum) == type(charge_ah_sum)
    assert type(time_sum) == type(discharge_ah_sum)
    
    
    

In [42]:
test_initialize_datasets()

In [ ]:
import import_data
import pandas as pd
import numpy as np

def test_single_pd_matlab_data():
    
    